In [ ]:
# If on the HPC, need to install everything in the requirements.txt file.

In [1]:
from utils import *
from models import *
from training import *

In [2]:
# Create the configs dictionary
configs = {

    ####### Model #######
    "model_name" : "resnet", # Model name
    "input_nc" : 1, # Number of input channels
    "output_nc" : 1, # Number of output channels
    "ngf" : 64, # Number of filters in first conv layer
    "num_blocks" : 9, # Number of residual blocks
    "norm_layer" : "BatchNorm3d", # Normalization layer
    "use_dropout" : False, # Dropout layers
    "padding_type" : "reflect", # Padding type
    
    ####### Training #######
    "n_epochs" : 100, # Number of epochs
    "loss" : "mse_loss", # Loss function
    "optimizer" : "Adam", # Optimizer
    "evaluation_metric" : "mse_loss", # Evaluation metric
    "save_best" : True, # Save best model
    "regularized" : False, # Regularization
    "vae" : False, # Variational autoencoder

    ####### Data #######
    "main_data_path" : "D:\\Brain-MINDS\\model_data", # Data path
    "training_log_path" : "D:\\Brain-MINDS\\tract_residuals\\logs\\resnet_log.csv", # Training log path
    "model_filename" : "D:\\Brain-MINDS\\tract_residuals\\models\\resnet.h5", # Model filename
    "batch_size" : 1, # Batch size
    "validation_batch_size" : 1, # Validation batch size
    
    ####### Parameters #######
    "initial_learning_rate" : 1e-04, # Initial learning rate
    "early_stopping_patience": 50, # Early stopping patience
    "decay_patience": 20, # Learning rate decay patience
    "decay_factor": 0.5, # Learning rate decay factor
    "min_learning_rate": 1e-08, # Minimum learning rate
    "save_last_n_models": 10, # Save last n models

    ####### Misc #######
    "skip_val" : True, # Skip validation

}

In [3]:
# Define the configuration path and save it as a .json file
config_path = os.path.join("configs", configs["model_name"] + ".json")

# Save the configuration
dump_json(configs, config_path)

In [4]:
# Load the configuration
configs = load_json(config_path)

# Define the metric to monitor based on whether we're skipping val or not
if configs["skip_val"]:
    metric_to_monitor = "loss"
else:
    metric_to_monitor = "val_loss"

# Define the groups
if configs["skip_val"]:
    groups = ("training",)
else:
    groups = ("training", "validation")

model_metrics = (configs["evaluation_metric"],)

run_pytorch_training(configs, configs["model_filename"], configs["training_log_path"], 
                     metric_to_monitor=metric_to_monitor,
                     bias=None)

Model is: ResnetEncoder
Criterion is: <function mse_loss at 0x00000203D94205E0>
Optimizer is: Adam
Batch: 0
B0: torch.Size([1, 256, 356, 230])
Residual: torch.Size([1, 256, 356, 230])
Injection center: torch.Size([1, 3])


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.00 GiB (GPU 0; 2.00 GiB total capacity; 376.33 MiB already allocated; 1.13 GiB free; 430.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF